Note: All links provided are optional reading unless I indicate otherwise.

# Introduction:

## 1. Lab Objective:

The purpose of this lab is to explore how to build phylogenetic trees and to think about the genetic diversity within different great ape genera. We will learn how to use select programs in Phylip to estimate a phylogenetic tree using distance-based methods. We will compare different trees made using different methods and assuming different models. We will gain experience visualizing trees and using them to understand evolutionary relationships. 

In this lab, the major workflow is to take genomic data sequenced for great apes, and 

    (A) choose random areas of the genome with enough data available, 
    (B) convert it into a format readable by the Phylip software, 
    (C) determine evolutionary distance assuming different mutation rate models, 
    (D) estimate a phylogenetic tree using those distances, 
    (E) visualize the tree,
    (F) compare different trees to each other. 

## 2. UNIX

By now, all of you should have successfully logged into the server and found the directory we will be working in:
```
/home/classes/B351_HEG_F19/
```
Everyone should have a folder they made as a pre-lab assignment. Each of you will work in these folders, as I have access to them and will be able to help you troubleshoot more easily. 

For the server, we don't have a nice interface like on our Macs and PCs with which to navigate the computer and ask it to perform functions. Thus, we use the [UNIX language](https://www.tutorialspoint.com/unix/index.htm), to tell the server what to do. You've already used some in the pre-lab; for instance:

1. cd </path/of/folders/> - cd means change directory to the folder specified in the path provided. That way, you can change to the folder you'd like to work in and you won't have to keep adding the path in front of every file you want to work with in a folder. 
2. mkdir <path/optional/name_of_folder> - mkdir means make a new folder in the directory you're in or in a different folder with the path specified. 
3. ls - ls means list all the files in the directory you are in. Typing "ls -lrth" adds options to list the files based on how recently it was edited, and it includes some file stats, including time edited and the size of the file. 

Here, I will mention a few more you will find useful throughout the lab:

4. less <filename> - This allows you to look or 'read' the contents of a file (but not edit). Press 'q' to quit. 
5. cp <oldfilename> <newfilename> - This allows you to copy a file to a new file name. It will keep the old file.
6. mv <oldfilename> <newfilename> - This allows you to rename your file to a new file name. It will not keep the old file (which is deleted permanently).
 

## 3. Data

The data we are using is from the **2013 Nature study by Prado-Martinez et al., “Great ape genetic diversity and population history” ([DOI:10.1038/nature12228](https://www.nature.com/articles/nature12228))**. This study contributed a large genomic dataset for all great ape species, particularly adding multiple genomes per species, so the diversity within great ape species could be studied. They used a variety of population genetic and phylogenetic methods to determine past population sizes, structure, and levels of genetic diversity within and between species. The authors [stored their data](https://eichlerlab.gs.washington.edu/greatape/), where they also provided some sample information as well. The dataset includes 88 individuals: 9 humans, 13 bonobos, 25 chimpanzees, 31 gorillas, and 10 orangutans. 

These data are downloaded onto the server and can be found at:

```
/home/classes/B351_HEG_F19/lab1/data/
```

There are four types of files:

1. `<Species name>.vcf.gz` - These are the actual data files, which are zipped (because they are very large when not compressed!) and have the Variant Call Format (VCF, see this [link](https://samtools.github.io/hts-specs/VCFv4.2.pdf)). They include SNP information for all the individuals within the labeled species or genera. All information on homologous sequences are already available, as data for all species are mapped to the human reference genome hg18. 
2. `<Species name>.vcf.gz.tbi` - I created these files. They use another software called `tabix` which indexes the VCF. That is, it makes a table of contents, so it is easy to call a particular set of lines by specifying the chromosome and position. 
3. `ape_indivs.txt` - I also created this file. Because the labels for each individual were very long (which would make reading it on a tree difficult in the final visualization), I shortened the IDs. The first letter indicates the genera: G=Gorilla; H=Homo; P=Pan; O=Pongo. Later letters refer to the species or subspecies. 
4. `Uncallable/` - In the uncallable folder, there are [BED files](https://useast.ensembl.org/info/website/upload/bed.html) and index files (*.tbi) for the BED files. These BED files indicate for which positions they could not retrieve any data. 
    
If I wrote the lab correctly, you should not have to mess with any of the above files (except perhaps ape_indivs.txt as a reference for yourself, but it is also in your tocopy/ folder). 



## 4. Python scripts

I have written two scripts to help us prepare files to run in the program Phylip. They are found in the `tocopy/` folder. A biologist who knows enough of a computing language to manipulate data files into formats readable by different software is actually fairly rare. Even mastering enough of a language to write these (usually simple) code goes a long way. I do not expect any of you to understand the code, but if you're interested in asking me how I did it, please do! 

1. `findgoodpos.py` - This python script takes a region you input and looks in the BED file to see which positions within that region were not sequenced. It outputs a list of the positions that were successfully sequenced. If any of the positions in the output file are not in the VCF file, we assume that they are fixed across all individuals. This is needed because without it, we would not have an accurate count of the total number of positions actually analyzed in the tree (we'd only have the SNPs). The arguments (that you put after the script) are `<# for chrom> <start position> <end position>`

2. `makeseqfile.py` - This python script takes the output file from findgoodpos.py, the VCF files, and the `ape_indivs.txt` file and uses them to create a sequence file that is readable in the Phylip software. The script treats each individual as haploid. Thus, for heterozygotes, one of the two possible alleles is randomly drawn. For sites fixed in all great apes, I randomly chose a nucleotide base for all individuals (the more accurate strategy is to find the reference hg18 data and put the reference allele, but the answer should be no different). 

## 5. PHYLIP

[Phylip](http://evolution.genetics.washington.edu/phylip.html) is actually a bundle of software all designed to build, visualize, and compare trees. While there are a few different software we could have used in Phylip, the two we will be focusing on are `dnadist` and `neighbor`. 

1. `dnadist` - This takes the input file we created using ```makeseqfile.py``` and it forms a distance matrix, assuming different evolutionary models. That is, it calculates all the pairwise distances and then applies the correction using different evolutionary models we discussed in class. We will focus on two models - Jukes-Cantor formula (JC69) and Kimura's 2-parameter (K2P or K89). The documentation is [here](http://evolution.genetics.washington.edu/phylip/doc/dnadist.html).
                                              
2. `neighbor` - After creating a distance matrix using `dnadist`, the software `neighbor` takes the distance matrix and determines the best tree for those distances using either Neighbor Joining or UPGMA. We will be making both trees. The documentation is [here](http://evolution.genetics.washington.edu/phylip/doc/neighbor.html)

There are software for visualizing trees, but we will actually use an online software [iTOL](https://itol.embl.de/), as it actually does a better job with creating trees with a large number of tips than the software in Phylip. Furthermore, it's interactive and allows you to try different settings (rooting, tip labeling, etc) easily. 

# Lab Pipeline

0. In your folder (e.g. `/home/classes/B351_HEG_F19/melinda/`), make a folder titled "`lab1_analysis/`" and copy the files from the `tocopy/` folder in `/home/classes/B351_HEG_F19/lab1/tocopy/`. You should now have the two scripts, the example files I ran using region chr1:1100000-1101000, and the `ape_indivs.txt` file. Make sure you are working in this new directory for the rest of class. The commands are below. 

```
mkdir /home/classes/B351_HEG_F19/<your first name>/lab1_analysis/
cp /home/classes/B351_HEG_F19/lab1/tocopy/* /home/classes/B351_HEG_F19/<your first name>/lab1_analysis/
cd /home/classes/B351_HEG_F19/<your first name>/lab1_analysis/

```

## A. Choose random areas of the genome with enough data available

Everyone in the class will be choosing different regions of the genome and assessing tree relationships. Pick a chromosome between 1 and 22 (we're ignoring sex chromosomes and mtDNA). If you're not sure which number, take a look at this [link](https://www.ncbi.nlm.nih.gov/grc/human/data?asm=NCBI36). It lists the total length of each chromosome in hg18. 

1. Pick a chromosome and position and add 1000 to the position. The position and 1000+position will be your start and end positions. Pick a few different ones to try, writing those numbers in the table on the worksheet. You will need at minimum two regions, though I recommend testing a few more, as not all regions may be suitable.

2. Determine the number of positions with data available using `findgoodpos.py`. To do this, type the following, substituting your numbers for the example positions. 

```
time ipython findgoodpos.py 1 1100000 1101000 
```
The `time` is added so you can see how long the script took to run. The `ipython` is telling the PY script to run using IPython. The arguments are chromosome number, start position, end position. The number of positions available for analysis will be printed to the screen, and an output file `c1_s1100000_e1101000.callablepos` will be created with all the positions to analyze. 

Record the chromosome, start position, end position, and number of positions available on the worksheet, including the ones you decided not to use. Decide which ones you want to make trees for, and star those. Pick two regions with at least 100 bases available to analyze in rest of lab.



## B. Convert data into a format readable by the Phylip software

3. After choosing the regions, use `makeseqfile.py` to make the sequence file that [the Phylip software can read](http://evolution.genetics.washington.edu/phylip/doc/main.html#inputfiles). Type the following, substituting your numbers for the example positions. 

```
time ipython makeseqfile.py 1 1100000 1101000 
```

This script has the same format for arguments as the previous script. It makes a text file with the sequence data in the input format requested by Phylip, with the name ```c1_s1100000_e1101000.seq```. For about 1,000 positions, it takes about 2-4 minutes. 

## C. Determine the evolutionary distance assuming different mutation rate models

4. With a data file in the correct format for Phylip, you now have to calculate evolutionary distances between samples. For each region, you will calculate the evolutionary distance assuming a Jukes-Cantor model and a Kimura 2-parameter model. Instructions are shown for the Jukes-Cantor model. Use `ls -lrth *seq` to list every SEQ file in your working directory. Copy the filename for which you want to estimate evolutionary distances. Then, type `dnadist` and press enter. 

5. The terminal will likely show:
```
dnadist: can't find input file "infile"
Please enter a new file name> 
```
To this, paste in your SEQ filename and press enter.

6. The program likes to default use files titled ```infile``` and ```outfile```, but we would rather be more specific. If the Terminal prompts:

```
dnadist: the file "outfile" that you wanted to
     use as output file already exists.
     Do you want to Replace it, Append to it,
     write to a new File, or Quit?
     (please type R, A, F, or Q) 
```

do the following:
Type 'F', press enter, input a different filename, and press enter again. I recommend ```c1_s1100000_e1101000.distjc``` as the format (and ```*.distk2p``` for the other mutation model).  

If it does not give the prompt, it will have written everything to ```outfile```. Then, pay attention to step #9. 

7. The next set of text shows the settings to estimate evolutionary distance. The only one we will focus on is the first one, titled 'Distance'. That needs to be adjusted to Jukes-Cantor. To do so, type 'D' and enter until it changes to "Jukes-Cantor". Then type 'Y' and enter. 

```
Nucleic acid sequence Distance Matrix program, version 3.697

Settings for this run:
  D  Distance (F84, Kimura, Jukes-Cantor, LogDet)?  F84
  G          Gamma distributed rates across sites?  No
  T                 Transition/transversion ratio?  2.0
  C            One category of substitution rates?  Yes
  W                         Use weights for sites?  No
  F                Use empirical base frequencies?  Yes
  L                       Form of distance matrix?  Square
  M                    Analyze multiple data sets?  No
  I                   Input sequences interleaved?  Yes
  0            Terminal type (IBM PC, ANSI, none)?  ANSI
  1             Print out the data at start of run  No
  2           Print indications of progress of run  Yes

  Y to accept these or type the letter for one to change
```

8. It should output a bunch of dots, ending with 

```
Distances written to file "c1_s1100000_e1101000.distjc"

Done.
```

If instead it says `outfile`, meaning you did not get prompted as shown in #6, then now type

`cp outfile c1_s1100000_e1101000.distjc`

This will copy the `outfile` to a file with the more specific naming format. 

9. Repeat steps #5-8 for the Kimura 2-parameter model, with the suggested label of `c1_s1100000_e1101000.distk2p`

10. Repeat steps #4-9 for the other region. In the end, you should have four DIST files - two `*.distjc` and two `*.distk2p`. Draw a flow chart on your worksheet to make sure you make every file. 


## D. Estimate a phylogenetic tree using those distances

Now, with distance matrices in hand, you will make both a neighbor joining (NJ) tree and an unweighted pair group method with arithmetic mean (UPGMA) tree. The instructions below are for an NJ tree, but you will repeat them, changing the settings for a UPGMA tree. 

11. Use `ls -lrth *dist*` to list all your distance matrix files. Copy the one you want to make into a tree. 

12. Type `neighbor` into the Terminal and press enter. When prompted, paste in your filename, `c1_s1100000_e1101000.distjc`. 

13. You will have the same error about the name of the `outfile` as above. Press 'F' and then enter. Put in your new name, e.g. `c1_s1100000_e1101000.distjc.njfile` (I suggest adding ".njfile" for an NJ outfile and ".upgmafile" for a UPGMA outfile). 

14. The settings for this software is:

```
Neighbor-Joining/UPGMA method version 3.697

Settings for this run:
  N       Neighbor-joining or UPGMA tree?  Neighbor-joining
  O                        Outgroup root?  No, use as outgroup species  1
  L         Lower-triangular data matrix?  No
  R         Upper-triangular data matrix?  No
  S                        Subreplicates?  No
  J     Randomize input order of species?  No. Use input order
  M           Analyze multiple data sets?  No
  0   Terminal type (IBM PC, ANSI, none)?  ANSI
  1    Print out the data at start of run  No
  2  Print indications of progress of run  Yes
  3                        Print out tree  Yes
  4       Write out trees onto tree file?  Yes


  Y to accept these or type the letter for one to change
```

The only setting to change should be the first one. Type 'N' and press enter to switch between UPGMA and NJ trees. Then type 'Y' and press enter. 

15. This software makes an `outfile` as well as a tree file, usually with the default name `outtree`. If it prompts you with:

```
neighbor: the file "outtree" that you wanted to
     use as output tree file already exists.
     Do you want to Replace it, Append to it,
     write to a new File, or Quit?
     (please type R, A, F, or Q)
```

Type 'F' and press enter, and then input your new name (recommended `*.njtree`, e.g. `c1_s1100000_e1101000.distjc.njtree`). If it does not, make sure to do the next step (#16).  

16. You will now have two new files. If one is named `outtree`, use `cp outtree c1_s1100000_e1101000.distjc.njtree` to copy to a file with a clearer description of what it is. 

17. a) The first file, (e.g. to look inside, type `less c1_s1100000_e1101000.distjc.njfile`) summarizes the information, providing the type of tree building algorithm you used, a text version of the tree (note that it is unrooted for an NJ tree though it may look rooted!), and branch lengths. The second file (e.g. to look inside, type `less c1_s1100000_e1101000.distjc.njtree`) is a bunch of numbers, colors, and parantheses that is hard to look at. This is a tree in Newick-style format (Wiki has a pretty good [description](https://en.wikipedia.org/wiki/Newick_format)), which stores the tree information concisely in text form. 

17. b) Note that for a UPGMA tree, a root is determined because it assumes an ultrametric tree. That is, it puts the root exactly where all the branch lengths from the root to the tip would be equal to each other. 

18. Repeat the above steps in part D for UPGMA, and then repeat everything for each distance matrix. Note, that for two regions, you have four distance matrices, and eight trees. In the worksheet, you are asked to draw a flow chart for this section as well. 

## E.  Visualize the tree

18. Finally, you have the information to visualize the tree. We will be doing this in [iTOL](https://itol.embl.de/). Go to this link on your web browser and click 'Upload' at the top. 

19. In your Terminal, open one of your tree files (e.g. ```less c1_s1100000_e1101000.distjc.njtree```). Copy everything in the file. Go to the upload page of iTOL on the web browser and paste it under the space for **"Tree text:"**. Click **Upload**

20. You now have a tree! There are a lot of different options to play around with. Note that the top left has zooming options. If you have a mouse, the scroller also allows you to play with the size of the tree. On the right, there are many controls. For **Display Mode**, determine whether you want a rooted or unrooted tree. Feel free to play around with other options, with the goal of having a tree that's easy to look at (and easy to tell what the tip labels are). 

21. When you are ready to export to PDF, click on the Export tab in the top right. Click on the **Format:** dropdown box and set it to PDF. Then, click **Export**. make sure **Export area:** is set to *Screen*. In the newly downloaded file, change the name to match your above format (e.g. `c1_s1100000_e1101000.distjc.njtree.pdf`). 

22. For some questions in the worksheet, you are asked to focus on properties of the tree you observe within a single genera. You can always re-paste your Newick tree into the iTOL website, but I suggest you pick a genera now, and for each tree you make, make a PDF for the whole tree **AND** for a zoomed in section containing individuals belonging to that genera. 

## F. Compare different trees to each other

You have now successfully made many trees! You will now compare the trees to each other, thinking of some major themes. For the post-lab, I will take the UPGMA tree using the K2P estimate to create a consensus tree. 

23. Take your UPGMA trees using the K2P estimate (i.e. two files; e.g. `c1_s1100000_e1101000.distk2p.upgmatree`) and copy it into a shared folder. If you ended up making more than two trees with these conditions, feel free to share them all - the more data, the better!

```
cp *.distk2p.upgmatree /home/classes/B351_HEG_F19/lab1/classdata/
```

I will be using the software ```consense``` to create a phylogeny indicating what the most common splits were (and listing the probability of each split). 

24. For the post-lab, you will also take a SEQ file for a larger (100k) region and make a tree. Choose one of the files found in `/home/classes/B351_HEG_F19/lab1/region100k`. Use the related questions to think about all the different trees you get and how they differ and are similar. Answer the questions on the rest of the worksheet. 
